<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
# from plan import plan; plan("classes", "surcharge")

### surcharge des opérateurs du langage

* le langage offre de nombreux opérateurs, e.g.
  * opérations arithmétiques : `+`, `-`, `*`, `\`
  * mais aussi : `x[i]`, `x()`, `x.attr`
* des builtins comme &nbsp;&nbsp;&nbsp;`print()`, `len()`
* des constructions syntaxiques &nbsp;&nbsp;&nbsp; `for i in x:`
* qui sont conçues sur les types de base  
  mais indéfinis, ou pauvres, sur les classes utilisateur

### surcharge des opérateurs

* ce dont il est question, c'est de donner les moyens  
  à chaque classe de mieux s'intégrer dans le langage

* comment ?  en redéfinissant des méthodes *spéciales:*
  * qui s'appellent toujours `__X__`
  * ou `X` est bien sûr en relation avec l'opération
  * e.g. `__add__` a effet sur l'opérateur `+`

### surcharge d’opérateurs

* la surcharge d’opérateurs est optionnelle
  * quoique: `__init__` et `__str__`
* toutes les méthodes que l’on peut surcharger sont décrites dans  
  https://docs.python.org/3/reference/datamodel.html#special-method-names

* lire cette documentation au moins une fois pour savoir tout ce que l’on peut surcharger
* **notamment** si vous êtes amenés à lire beaucoup de code

### les plus communs

* nous avons déjà vu `__init__`
* il est très fréquent de redéfinir aussi
  * `__repr__`: pour redéfinir `repr()`
  * `__str__`: pour redéfinir `str(x)` et `print()`

In [ ]:
# en l'absence de redéfinition 
# la présentation est aride
# et identique pour les deux modes repr/str

class Dumb:
    pass

a = Dumb()

In [ ]:
# __repr__()
a

In [ ]:
# __str__()
str(a)

In [ ]:
# __repr__()
repr(a)

In [ ]:
# __str__()
print(a)

### seulement `__repr__` (i.e. pas `__str__`)

il est assez fréquent de ne redéfinir que `__repr__`

In [ ]:
class R:
    def __init__(self, atom):
        self.atom = atom

    def __repr__(self):
        return "[R {}]".format(self.atom)

a = R('seulement repr')

In [ ]:
# __repr__()
a

In [ ]:
# __str__()
str(a)

In [ ]:
# __repr__()
repr(a)

In [ ]:
# __str__()
print(a)

# `__str__` et `__repr__`

* il faut savoir que
  * si `__repr__` est défini, et pas `__str__`
  * alors on fait comme si `__str__ = __repr__`
* et aussi que `__str__` sur les containers (list, ...)
  * appelle en fait `__repr__` sur les contenus
  * ceci pour éviter les récursions infinies...

* dans l'esprit:
  * `__repr__` est censé être non-ambigu
  * et `__str__` est censé être joli
* mais ce n'est pas toujours facile à suivre 

# `__str__` et `__repr__`

In [ ]:
# si maintenant on définit aussi `__str__`
class R:
    def __init__(self, atom):
        self.atom = atom
    def __repr__(self):
        return "[R {}]".format(self.atom)
    def __str__(self):
        return str(self.atom)

b = R('les deux')

In [ ]:
# __repr__()
b

In [ ]:
# __str__()
str(b)

In [ ]:
# __repr__()
repr(b)

In [ ]:
# __str__()
print(b)

### surcharge d’opérateurs numériques

In [ ]:
# pour redéfinir l'addition, sans surprise on surcharge __add__ 
# ici on choisit un comportement folklorique
# qui fait une espèce de concaténation

class C():
    
    def __init__(self, value):
        self.value = value

    def __add__(self, operand):
        # l'addition crée un nouvel objet
        return C(self.value + '-' + operand.value)

In [ ]:
x, y, z = C('alice'), C('bob'), C('eve')
s = x + y + z
s.value

##### c'est un début, mais

In [ ]:
# on ne peut pas additionner C avec un str
try:
    C('abc') + 'def'
    
except AttributeError as e:
    print("OOOPS", e)

##### et aussi

In [ ]:
# on ne peut pas non plus additionner lorsque
# l'instance de C est à gauche dans l'addition

try:
    'abc' + C('abc')
    
except TypeError as e:
    print("OOOPS", e)

# opérateurs binaires

pour faire proprement, il faut

* envisager le mélange avec d'autres types
  * polymorphisme :
  * *C + str, C + int, C + float*…
* envisager qu'un objet de notre classe peut être
  * commutatif :
  * *str + C, int + C, float + C*

### polymorphe - v1

##### une première amélioration naïve

In [ ]:
# sans définir __str__
class C1():

    def __init__(self, value):
        self.value = value

    def __add__(self, operand):
        # soyons plus subtils
        if isinstance(operand, C):
            ajout = operand.value
        else:
            ajout = str(operand)

        # le résultat est un nouvel objet
        return C1(self.value + '-' + ajout)

In [ ]:
# maintenant on peut ajouter un C avec un str
(C1('alice') + 'bob').value

### polymorphe - v2

##### c'est beaucoup plus simple si on redéfinit `str()`

In [ ]:
# une deuxième amélioration 

class C2():
    def __init__(self, value):
        self.value = value
    # ici on redéfinit __str__
    def __str__(self):
        return str(self.value)
    def __add__(self, operand):
        # comme on a redéfini __str__, 
        # on peut écrire tout simplement :
        return C2(self.value + '-' + str(operand))

In [ ]:
(C2('alice') + 'bob').value

In [ ]:
# mais par contre la présentation n'est toujours pas très jolie
C2('alice') + 'bob'

##### vous voyez pourquoi on redéfinit souvent `__repr__`

### polymorphisme - v2 

In [ ]:
# un autre souci avec cette approche
# c'est dans le cas d'une sous-classe
class Sub2(C2):
    pass

In [ ]:
# si on additionne deux instances 
# de la sous-classe
s2 = Sub2('alice') + Sub2('bob')

In [ ]:
# on obtient un objet .. de la superclasse
type(s2)

### polymorphe - v3

on peut améliorer encore un peu

* définissons `__repr__` plutôt que `__str__`
* et aussi créons un objet **de la même classe**
  * plutôt que de câbler en dur le nom de notre classe
  * comme ça nos sous-classes seront plus à l'aise

In [ ]:
# une amélioration plus 'subclass-friendly'

class C3():
    def __init__(self, value):
        self.value = value

    # c'est plus simple de définir __repr__
    def __repr__(self):
        return str(self.value)

    def __add__(self, operand):
        # cette forme-là permet à une sous-classe
        # de créer des instances à elle plutôt que
        # forcément un C3
        return self.__class__(self.value + '-' + str(operand))

In [ ]:
(C3('alice') + 'bob').value

In [ ]:
# et cette fois
s = C3('alice') + 'bob'
s

In [ ]:
# on a bien un objet 
# de la classe C3
type(s)

# opérateurs binaires : à droite

* quand on fait `C('bob') + 'alice'
  * c'est à l'opérande gauche
  * qu'on envoie la méthode `__add__`
* si on veut pouvoir ajouter dans l'autre sens
  * c'est-à-dire `'bob' + C('alice')`
* il suffit de redéfinir `__raddr__`
  * le `r` voulant dire *right*
  * pour quand le sujet de la méthode est à droite 

In [ ]:
# opérateurs à droite
class CR():
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)

    def __add__(self, rightop):
        return self.__class__(self.value + '-' + str(rightop))

    # dans le cas d'une algèbre commutative on peut juste faire
    # __raddr__ = __addr__
    # mais ici bien la concaténation n'est pas commutative
    def __radd__(self, leftop):
        return self.__class__(str(leftop) + '-' + self.value)

In [ ]:
'bob' + CR('alice')

In [ ]:
# avec cette version aboutie de notre classe
# on a tous les avantages recherchés
# notamment utilisable avec une sous-classe

class SubCR(CR):
    pass

In [ ]:
o1 = SubCR('sousobj') + 'string'
o1

In [ ]:
o2 = 'string' + SubCR('sousobj')
o2

In [ ]:
type(o1), type(o2)

### protocoles

* certaines constructions du langage
  * sont simples et n'utilisent qu'**une seule** spéciale
  * ex. `__len__`
* d'autres reposent sur **plusieurs méthodes spéciales**
  * par exemple `x[]` utilise:  
    `__getitem__()` pour les références  
    `__setitem__()` pour les affectations  
    `__delitem__()` pour `del x[]`  
    `__missing__()` pour les défauts de clé

* ou sur **une parmi plusieurs**
  * par exemple `i in x` peut fonctionner avec  
    `__contains__()` ou  
    `__iter__()` ou  
    `__getitem__()`

### protocoles et vocabulaire

* dans le cadre du *duck typing*
* il est fréquent de faire référence 
  * à des grandes familles d'objet
  * comme e.g. séquences, itérables, callables, ...
* par exemple une *séquence*
  * doit implémenter `x[i]` avec i entier
  * et `len(x)`

### le protocole itérable

* un objet est itérable
  * lorsque qu'on peut écrire `for i in x`
* deux moyens
  * une séquence
  * implémenter `__iter__()`
  * qui doit retourner un itérateur

### le protocole iterator

* un objet est un itérateur si
  * il implémente `__next__()`
  * qui retourne l'objet suivant
  * ou lève l'exception `StopIteration`
  * et il implémente `__iter__` qui renvoie `self`
* un itérateur est donc toujours itérable
* une fonction génératrice renvoie un itérateur

### itérable avec générateur

In [ ]:
from itertools import count

class Iterable:
    """itérer les carrés <= n"""
    def __init__(self, n):
        self.n = n

    # il est pratique d'utiliser un générateur
    # pour implémenter __iter__
    def __iter__(self):
        for i in count():
            square = i ** 2
            if square >= 20:
                return
            yield square

In [ ]:
# équivalent à 

def IterableGenerator(n):
    for i in count():
        square = i ** 2
        if square >= 20:
            return
        yield square        

***

In [ ]:
for n in Iterable(20):
    print(n)

In [ ]:
for n in IterableGenerator(20):
    print(n)

### callables

* un objet est callable si on peut évaluer `x()`
* pas réservé aux fonctions et aux classes
* les instances d'une classe 
  * qui implémente `__call__`
  * sont callables également
* confusion fréquente
  * appeler la classe `c = C()` : utilise `__init__`
  * appeler l'instance `c()` : utilise `__call__`

### callables

In [ ]:
class SumOffset:
    """"
    chaque instance possède un offset
    lorsque l'instance est appelée elle fait la 
    somme de ses arguments plus l'offset
    """
    def __init__(self, offset):
        print("init")
        self.offset = offset
        
    def __call__(self, *args):
        print("calling..")
        return sum(args) + self.offset

In [ ]:
# cette instance est un callable
# elle se comporte comme une fonction
# qui rend 100 + sigma(args)
additionneur100 = SumOffset(100)

In [ ]:
# quand on l'appelle
additionneur100(1, 2, 3)

In [ ]:
additionneur100(1000, 2000)

### exemple d’autres surcharges d’opérateurs

* `__lt__`, `__gt__`, `__le__`, `__ge__`, `__eq__`, `__ne__`  
  *resp.* &nbsp; `A<B`, &nbsp; `A>B`, &nbsp; `A<=B`, &nbsp; `A>=B`, &nbsp; `A==B`, &nbsp; `A!=B`

* `__bool__` : appelé pour tester si un objet est vrai ou faux
* `__len__`: redéfinir `len(x)` 
* `__getattr__`, `__slot__`, `__getattribute__`  
  impliqués dans le protocole de recherche d'attributs

* ... liste très très complète

# attributs privés

##### Rappels

* pas de notion d'attributs protégé / privé en Python
  * on peut accéder à n’importe quels attributs d’une classe
* on représente un attribut privé avec une simple convention de nommage 
  * les attributs qui commencent par `_` sont considérés comme privés

### trois types d’attributs privés réservés

* les attributs `_*` ne sont pas importés par `from module import *`
* les attributs `__*__` sont les attributs privés définis par Python. On ne doit pas nommer nos propres attributs privés avec cette convention
* les attributs `__*` (sans `__` à la fin) définis dans une classe sont automatiquement renommés à la compilation (`__spam` dans la classe Ham devient `_Ham__spam`). On appelle cela *name mangling*

### name mangling

In [ ]:
# utiliser le name mangling pour 
# un attribut privé qui
# ne doit pas être modifié
# par une sous-classe par accident
class A():
    def __init__(self):
        self.__a = "dans A"
    def __str__(self):
        return self.__a

class B(A):
    def __init__(self):
        A.__init__(self)
        # on est sûr de n'interférer avec personne
        self.__a = "dans B" 

In [ ]:
b = B()
print(b)

In [ ]:
print(b.__dict__)

In [ ]:
try:
    b.__a
except Exception as exc:
    print(f"OOPS {type(exc)} {exc}")

# classes imbriquées

* si une classe `A` définit une autre classe `B`,
  * on peut créer des instances de `B`
  * par la classe `A` avec `A.B()`
  * ou par une de ses instances `A().B()`
* en effet, on peut y accéder aussi bien
  * directement par la classe `A`, ou
  * par une instance grâce à l’héritage

In [ ]:
class A:
    class B:
        pass

In [ ]:
A.B()

In [ ]:
A().B()

# quand utiliser la POO en Python ?

* utilisation de base
  * sans héritage mais avec encapsulation
  * bénéfice de grouper le code et les données
  * dans des espaces de noms étanches
* héritage
  * demande en général un peu de conception en amont
  * ce n'est **pas forcément le plus gros bénéfice**
  * sauf à mon humble avis pour la surcharge des opérateurs
  * qui s'avère vite utile - homéopatiquement
  * une fois qu'on a passé le barrage d'entrée

# modules ou classes ?

* utiliser une classe dès qu'on a besoin
  * de créer des instances multiples
  * d'exploiter la notion d’héritage
* se contenter d'un module si on veut simplement
  * isoler des espaces de nommages
  * créer des méthodes statiques
  * factoriser du code
* opinion personnelle
  * ne me souviens pas d'avoir écrit un module sans classe
  * ou alors pour grouper quelques helpers → `utils.py` 
  * ce qui ne veut pas dire qu'un module ne contient jamais de fonction

# pour réutiliser du code en python

* fonctions
  * pas d'état après exécution
* modules
  * garde l'état
  * une seule instance par programme
* **classes**
  * **instances multiples**
  * **chacune garde l'état**
  * **héritage**

# pour conclure

* *design patterns*
  * quelques idées assez génériques éprouvées
  * pas de magie ou de théorie complexe dans les design patterns
  * liste de recettes empiriques des auteurs
  * propices à l'exploitation de l'héritage

![design patterns](pictures/book-design-patterns.png)

# partie optionnelle

### utiliser un opérateur ou la méthode `__x__` ?

* en version courte: **utilisez les opérateurs**
* résultat est équivalent parce que le même code est utilisé, mais la durée d’exécution peut différer à cause d’optimisations de l’interpréteur
* l’interpréteur va optimiser l’appel à la fonction lors de utilisation des opérateurs, mais pas lors de l’appel direct à la méthode `__x__`
  * spécifique à CPython, il faut donc tester ce comportement pour les autres implémentations de Python

### un peu de profiling

* voyez le module `timeit`
  * https://docs.python.org/3/library/timeit.html

In [ ]:
from timeit import timeit

### utiliser un opérateur ou la méthode `__x__` ?

In [ ]:
timeit(setup = "L = range(1000)", number = 100000, stmt = "1000 in L")

In [ ]:
timeit(setup = "L = range(1000)", number = 100000,
       stmt = "L.__contains__(1000)")

```
timeit(setup = "L = range(1000)", number = 100000000, stmt = "0 in L")
9.534808637050446
timeit(setup = "L = range(1000)", number = 100000000, stmt = "L.__contains__(0)")
19.80092801299179
```

### profiling et notebooks

beaucoup de [fonctionnalités très intéressantes dans les *magic* IPython](https://ipython.readthedocs.io/en/stable/interactive/magics.html), comme `%timeit`

In [ ]:
L = range(1000)
%timeit -n 100000 0 in L

In [ ]:
L = range(1000)
%timeit -n 100000 L.__contains__(0)